In [1]:
import pandas as pd

In [2]:
comments = pd.read_csv("places_reviews (3) (1) (2).csv")
comments =  comments[[ "text", "language", "place_id" ]]
comments.dropna(inplace=True)
comments = comments[ comments.language=='es']
comments.drop(['language'], inplace=True, axis=1)
comments

,text,place_id
0,"Higiene, medidas de sana distancia, excelente ...",ChIJ_6PxIP_-0YURb-iv-Yq3gPM
1,Buen lugar para comer y pasar el rato,ChIJ_6PxIP_-0YURb-iv-Yq3gPM
2,Súper delicioso todo y muy económico 🥰❤️❤️,ChIJ_6PxIP_-0YURb-iv-Yq3gPM
3,"Muy buen servicio, costos un poco elevados",ChIJ_6PxIP_-0YURb-iv-Yq3gPM
4,"Comida rica, buen precio, atención normal, la ...",ChIJ_6PxIP_-0YURb-iv-Yq3gPM
...,...,...
9178,Good place to eat traditional food from Jalisc...,ChIJRU2FLk__0YURikrh-RZMung
9179,"Meseros siempre con mala cara, tortas ahogadas...",ChIJRU2FLk__0YURikrh-RZMung
9180,Muy ricas las tortas ahogadas 🤤🤤🤤,ChIJRU2FLk__0YURikrh-RZMung
9181,Riquísimo\nTortas ahogadas\nTacos dorados\nTac...,ChIJRU2FLk__0YURikrh-RZMung


In [3]:
import re
comments['texto'] = comments['text'].map(lambda x: re.sub('[,\.!?]', '', x))
comments['texto'] = comments['texto'].map(lambda x: x.lower())
comments

,text,place_id,texto
0,"Higiene, medidas de sana distancia, excelente ...",ChIJ_6PxIP_-0YURb-iv-Yq3gPM,higiene medidas de sana distancia excelente ca...
1,Buen lugar para comer y pasar el rato,ChIJ_6PxIP_-0YURb-iv-Yq3gPM,buen lugar para comer y pasar el rato
2,Súper delicioso todo y muy económico 🥰❤️❤️,ChIJ_6PxIP_-0YURb-iv-Yq3gPM,súper delicioso todo y muy económico 🥰❤️❤️
3,"Muy buen servicio, costos un poco elevados",ChIJ_6PxIP_-0YURb-iv-Yq3gPM,muy buen servicio costos un poco elevados
4,"Comida rica, buen precio, atención normal, la ...",ChIJ_6PxIP_-0YURb-iv-Yq3gPM,comida rica buen precio atención normal la bar...
...,...,...,...
9178,Good place to eat traditional food from Jalisc...,ChIJRU2FLk__0YURikrh-RZMung,good place to eat traditional food from jalisc...
9179,"Meseros siempre con mala cara, tortas ahogadas...",ChIJRU2FLk__0YURikrh-RZMung,meseros siempre con mala cara tortas ahogadas ...
9180,Muy ricas las tortas ahogadas 🤤🤤🤤,ChIJRU2FLk__0YURikrh-RZMung,muy ricas las tortas ahogadas 🤤🤤🤤
9181,Riquísimo\nTortas ahogadas\nTacos dorados\nTac...,ChIJRU2FLk__0YURikrh-RZMung,riquísimo\ntortas ahogadas\ntacos dorados\ntac...


In [4]:
import gensim
from gensim.utils import simple_preprocess
import nltk
from nltk.stem.porter import PorterStemmer

nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = stopwords.words('spanish')



[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/antonio/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [15]:
p_stemmer = PorterStemmer()

def sent_to_words(sentences):
    for sentence in sentences:
        # deacc=True removes punctuations
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))
def remove_stopwords(texts):
    return [[ p_stemmer.stem(word) for word in simple_preprocess(str(doc)) 
             if word not in stop_words] for doc in texts]




In [16]:
data = comments.texto.values.tolist()
data_words = list(sent_to_words(data))
# remove stop words
data_words = remove_stopwords(data_words)
print(data_words[:1][0][:30])

['higien', 'medida', 'sana', 'distancia', 'excelent', 'calidad', 'alimento', 'maravillosa', 'atencion', 'vale', 'pena']


In [17]:
import gensim.corpora as corpora
# Create Dictionary
id2word = corpora.Dictionary(data_words)
id2word.filter_extremes(no_below=20, no_above=0.5)
texts = data_words
corpus = [id2word.doc2bow(text) for text in texts]


In [18]:
from pprint import pprint
# number of topics
num_topics = 4
# Build LDA model
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=num_topics)
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.040*"lugar" + 0.029*"buen" + 0.022*"bien" + 0.022*"mejor" + '
  '0.020*"comida" + 0.019*"atencion" + 0.019*"servicio" + 0.018*"excelent" + '
  '0.018*"sabor" + 0.015*"ma"'),
 (1,
  '0.058*"excelent" + 0.054*"comida" + 0.049*"lugar" + 0.046*"buena" + '
  '0.031*"buen" + 0.028*"servicio" + 0.028*"atencion" + 0.025*"delicioso" + '
  '0.025*"precio" + 0.019*"rica"'),
 (2,
  '0.055*"servicio" + 0.044*"comida" + 0.035*"buen" + 0.034*"rico" + '
  '0.033*"excelent" + 0.032*"precio" + 0.027*"bueno" + 0.023*"lugar" + '
  '0.018*"calidad" + 0.018*"sabor"'),
 (3,
  '0.060*"excelent" + 0.056*"lugar" + 0.038*"comida" + 0.036*"atencion" + '
  '0.027*"buena" + 0.026*"sabor" + 0.023*"servicio" + 0.019*"bueno" + '
  '0.016*"recomend" + 0.015*"comer"')]


In [19]:
lda_model.print_topics()

[(0,
  '0.040*"lugar" + 0.029*"buen" + 0.022*"bien" + 0.022*"mejor" + 0.020*"comida" + 0.019*"atencion" + 0.019*"servicio" + 0.018*"excelent" + 0.018*"sabor" + 0.015*"ma"'),
 (1,
  '0.058*"excelent" + 0.054*"comida" + 0.049*"lugar" + 0.046*"buena" + 0.031*"buen" + 0.028*"servicio" + 0.028*"atencion" + 0.025*"delicioso" + 0.025*"precio" + 0.019*"rica"'),
 (2,
  '0.055*"servicio" + 0.044*"comida" + 0.035*"buen" + 0.034*"rico" + 0.033*"excelent" + 0.032*"precio" + 0.027*"bueno" + 0.023*"lugar" + 0.018*"calidad" + 0.018*"sabor"'),
 (3,
  '0.060*"excelent" + 0.056*"lugar" + 0.038*"comida" + 0.036*"atencion" + 0.027*"buena" + 0.026*"sabor" + 0.023*"servicio" + 0.019*"bueno" + 0.016*"recomend" + 0.015*"comer"')]

In [20]:
from pprint import pprint
# number of topics
num_topics = 4
# Build LDA model
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=num_topics)
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.070*"lugar" + 0.056*"excelent" + 0.038*"atencion" + 0.031*"rico" + '
  '0.029*"buena" + 0.025*"buen" + 0.025*"precio" + 0.020*"servicio" + '
  '0.020*"bueno" + 0.019*"comida"'),
 (1,
  '0.051*"comida" + 0.047*"lugar" + 0.041*"excelent" + 0.035*"servicio" + '
  '0.025*"buen" + 0.024*"bien" + 0.023*"mejor" + 0.019*"buena" + 0.019*"bueno" '
  '+ 0.018*"atencion"'),
 (2,
  '0.054*"servicio" + 0.037*"lugar" + 0.037*"excelent" + 0.034*"buen" + '
  '0.032*"rico" + 0.031*"delicioso" + 0.020*"comida" + 0.020*"mejor" + '
  '0.014*"sabor" + 0.014*"super"'),
 (3,
  '0.067*"comida" + 0.047*"excelent" + 0.043*"buena" + 0.035*"sabor" + '
  '0.032*"atencion" + 0.029*"precio" + 0.025*"rica" + 0.022*"lugar" + '
  '0.022*"buen" + 0.017*"recomend"')]


In [21]:
import pyLDAvis.gensim_models
import pickle 
import pyLDAvis
import os
# Visualize the topics
pyLDAvis.enable_notebook()
LDAvis_data_filepath = os.path.join('./results/ldavis_prepared_'+str(num_topics))
# # this is a bit time consuming - make the if statement True
# # if you want to execute visualization prep yourself
if 1 == 1:
    LDAvis_prepared = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word)
    with open(LDAvis_data_filepath, 'wb') as f:
        pickle.dump(LDAvis_prepared, f)
# load the pre-prepared pyLDAvis data from disk
with open(LDAvis_data_filepath, 'rb') as f:
    LDAvis_prepared = pickle.load(f)
pyLDAvis.save_html(LDAvis_prepared, './results/ldavis_prepared_'+ str(num_topics) +'.html')
LDAvis_prepared


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3      0.048296  0.011608       1        1  29.026821
2     -0.046147  0.008016       2        1  24.571737
0      0.002261 -0.041558       3        1  23.608237
1     -0.004411  0.021934       4        1  22.793205, topic_info=           Term         Freq        Total Category  logprob  loglift
13        lugar  2131.000000  2131.000000  Default  30.0000  30.0000
82         rico   937.000000   937.000000  Default  29.0000  29.0000
25       comida  2020.000000  2020.000000  Default  28.0000  28.0000
21     servicio  1512.000000  1512.000000  Default  27.0000  27.0000
16    delicioso   691.000000   691.000000  Default  26.0000  26.0000
..          ...          ...          ...      ...      ...      ...
0      alimento    65.509739   257.824946   Topic4  -5.1520   0.1086
28           ma    75.968905   477.436853   Topic4  -5.0039  -0.3594
126  restaurant    59.706066   229.927256   Topic4  -5.2448   0.1304
71     platillo    68.046576   492.940044   Topic4  -5.1140  -0.5015
61        sabor    62.536226   859.329759   Topic4  -5.1984  -1.1417

[274 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
117       1  0.508543     acces
117       2  0.093970     acces
117       3  0.315075     acces
117       4  0.077387     acces
151       1  0.160020  acogedor
...     ...       ...       ...
243       4  0.519643     vista
289       1  0.533809    volver
289       2  0.092836    volver
289       3  0.116046    volver
289       4  0.255300    volver

[689 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 3, 1, 2])